# 1 Preparação do ambiente

Preparação do ambiente para execução do script.

## 1.1 Tempo inicial de processamento

In [ ]:
# Import das bibliotecas.
import time
import datetime

# Marca o tempo de início do processamento
inicioProcessamento = time.time()

In [ ]:
def salvar(nomeArquivo,texto):                       
    """
      Salva um texto em arquivo.
     
      Parâmetros:
        `nome_arquivo` - Nome do arquivo a ser salvo.
        `texto` - Texto a ser salvo.     
    """

    arquivo = open(nomeArquivo, "w")
    arquivo.write(str(texto))
    arquivo.close()

Função auxiliar para formatar o tempo como `hh: mm: ss`

In [ ]:
# Import das bibliotecas.
import time
import datetime

def formataTempo(tempo):
    """
      Pega a tempo em segundos e retorna uma string hh:mm:ss
    """
    # Arredonda para o segundo mais próximo.
    tempoArredondado = int(round((tempo)))
    
    # Formata como hh:mm:ss
    return str(datetime.timedelta(seconds=tempoArredondado))    

Biblioteca de limpeza de tela


In [ ]:
# Import das bibliotecas.
from IPython.display import clear_output

## 1.2 Tratamento de logs

In [ ]:
# Import das bibliotecas.
import logging # Biblioteca de logging

# Formatando a mensagem de logging
logging.basicConfig(format="%(asctime)s : %(levelname)s : %(message)s", level=logging.INFO)

## 1.3 Identificando o ambiente Colab

In [ ]:
# Import das bibliotecas.
import sys # Biblioteca para acessar módulos do sistema

# Se estiver executando no Google Colaboratory
# Retorna true ou false se estiver no Google Colaboratory
IN_COLAB = "google.colab" in sys.modules

## 1.4 Monta uma pasta no google drive para carregar os arquivos de dados.

In [ ]:
# import necessário
from google.colab import drive

# Monta o drive na pasta especificada
drive.mount("/content/drive")     

Mounted at /content/drive


Instalação

## 1.5 Instalação SPARQLWrapper

In [ ]:
!pip install SPARQLWrapper deep-translator

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 128 kB 51.5 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


# 2 Parametrização

## Nome do diretório dos arquivos de dados

In [ ]:
# Diretório do cohebert
DIRETORIO_CLINIASSIST = "DADOS_CLINICOS_PLANO_BERTIMBAU"

## Define o caminho para os arquivos de dados

In [ ]:
# Diretório local para os arquivos pré-processados
DIRETORIO_LOCAL = "/content/" + DIRETORIO_CLINIASSIST + "/"

# Diretório no google drive com os arquivos pré-processados
DIRETORIO_DRIVE = "/content/drive/MyDrive/Colab Notebooks/Data/" + DIRETORIO_CLINIASSIST + "/"

## Especifica os nomes dos arquivos de dados a serem lidos



In [ ]:
# Nome do arquivo
NOMEARQUIVOORIGINAL = "original.csv"
NOMEARQUIVOORIGINALCOMPACTADO = "original.zip"

In [ ]:
# Nome do arquivo com os dados de NER
ARQUIVO_CSV_TIPOS_DBPEDIA = 'tipos_dbpedia.csv'

## Especifica se deve carregar um arquivo de tipos

In [ ]:
# Caso Positivo o processo irá tentar carregar um arquivo com os tipos
# do drive e edita-lo inserindo as novas palavras colsultadas
LER_CSV_TIPOS_DBPEDIA_DO_DRIVE = True

# 3 Função NER DBpedia

## 3.1 Import da biblioteca SPARQLWrapper

In [ ]:
# importando biblioteca responsável por fazer consultas em SPARQL na DBpedia
from SPARQLWrapper import SPARQLWrapper, JSON
import re

sparql = SPARQLWrapper("http://dbpedia.org/sparql")

## 3.2 Import da Biblioteca de tradução

In [ ]:
from deep_translator import GoogleTranslator

In [ ]:
print(GoogleTranslator().get_supported_languages(as_dict=True))

{'afrikaans': 'af', 'albanian': 'sq', 'amharic': 'am', 'arabic': 'ar', 'armenian': 'hy', 'azerbaijani': 'az', 'basque': 'eu', 'belarusian': 'be', 'bengali': 'bn', 'bosnian': 'bs', 'bulgarian': 'bg', 'catalan': 'ca', 'cebuano': 'ceb', 'chichewa': 'ny', 'chinese (simplified)': 'zh-CN', 'chinese (traditional)': 'zh-TW', 'corsican': 'co', 'croatian': 'hr', 'czech': 'cs', 'danish': 'da', 'dutch': 'nl', 'english': 'en', 'esperanto': 'eo', 'estonian': 'et', 'filipino': 'tl', 'finnish': 'fi', 'french': 'fr', 'frisian': 'fy', 'galician': 'gl', 'georgian': 'ka', 'german': 'de', 'greek': 'el', 'gujarati': 'gu', 'haitian creole': 'ht', 'hausa': 'ha', 'hawaiian': 'haw', 'hebrew': 'iw', 'hindi': 'hi', 'hmong': 'hmn', 'hungarian': 'hu', 'icelandic': 'is', 'igbo': 'ig', 'indonesian': 'id', 'irish': 'ga', 'italian': 'it', 'japanese': 'ja', 'javanese': 'jw', 'kannada': 'kn', 'kazakh': 'kk', 'khmer': 'km', 'kinyarwanda': 'rw', 'korean': 'ko', 'kurdish': 'ku', 'kyrgyz': 'ky', 'lao': 'lo', 'latin': 'la', '

## 3.3 Definição dos tipos a serem observados

In [ ]:
# We annotated four types of entities corresponding to UMLS semantic groups (SG) of pathologies (DISO), anatomic entities (ANAT),
# biochemical or pharmacological substances (CHEM) and lab tests, diagnostic or therapeutic procedures (PROC) 
# Nós adicionamos mais 1 categoria para as especialidades médicas (SPEC)
from enum import Enum

class SG(Enum):
  CHEM = 1
  DISO = 2
  ANAT = 3
  PROC = 4
  SPEC = 5
  FOOD = 6

# Dicionário de tipos correspondentes aos grupos semanticos observados
TIPOS_OBSERVADOS = {
  'Drug' : 1,
  'ChemicalSubstance': 1,
  'ChemicalCompound': 1,
  'Disease' : 2,
  'AnatomicalStructure' : 3,
  'Work': 4,
  'MedicalSpecialty' : 5,
  'PersonFunction' : 5,
  'Food' : 6
}
#Profession

## 3.4 Funções de consulta ao DBpedia

In [ ]:
# Dicionário onde serão guardados os tipos retornados pela DBpedia de todass as palavras
TODOS_OS_TIPOS = dict()

In [ ]:
def getListaTiposDBpedia(sentenca):
  """
    Função responsável por consultar cada palavra da sentença enviada como parâmetro
    na base de ontologias da DBpedia e verificar se os tipos de entidade retornados
    estão dentro dos tipos a serem observados

    sentença {String} -- sentença com as palavras a serem consultadas
  """
  
  # remove parenteses e outros caracteres especiais
  sentenca = re.sub(u"[^\w\- ]", '', sentenca)
  
  # apenas palavras com tamanho maior que 2 e que não são apenas dígitos
  palavras = [x.capitalize() for x in sentenca.split() if (len(x) > 2 and not x.isdigit())]

  for palavra in palavras:
    
    # remove hifens nas estremidades das palavras
    palavra = re.sub(r"^\W|\W\Z", "", palavra)
    
    # Coloca a palavra no dicionário para não fazer uma segunda consulta com a mesma palavra
    if not palavra in TODOS_OS_TIPOS:
      TODOS_OS_TIPOS[palavra] = []

      # Query usada para fazer a pesquisa na DBPedia
      """
      select distinct ?i, ?type
        where {{ ?i rdfs:label "Cetoprofeno"@pt ; 
        a ?type . 
        FILTER (strstarts(str(?type), str("http://dbpedia.org/ontology/"))) 
        FILTER (strstarts(str(?i), str("http://dbpedia.org/resource/Ketoprofen"))) --palavra em ingles. 
      }}
      """
      query = f'select distinct ?type where {{ ?i rdfs:label "{palavra}"@pt ; a ?type . FILTER (strstarts(str(?type), str("http://dbpedia.org/ontology/"))) }}'
      
      sparql.setQuery(query)
      
      # Configura o retorno para o formato JSON
      sparql.setReturnFormat(JSON)
      
      # Realiza a consulta
      dbpedia_return_dict = sparql.query().convert()
      
      # Obtem a lista de "types" retornados pela consulta
      return_list = dbpedia_return_dict['results']['bindings']

      # Verifica se existem resultados
      if return_list:
        
        tipos_dbpedia_retornados = [result['type']['value'].split('/')[-1] for result in return_list]

        tipos_obsdervados_encontrados = [tipo for tipo in tipos_dbpedia_retornados if tipo in TIPOS_OBSERVADOS]

        tipos_codificados = []
        for tipo in tipos_obsdervados_encontrados:
          if TIPOS_OBSERVADOS[tipo] not in tipos_codificados:
            tipos_codificados.append(TIPOS_OBSERVADOS[tipo])
        
        # Se foi identificado apenas 1 tipo observado no retorno
        if len(tipos_codificados) == 1:
          
          TODOS_OS_TIPOS[palavra] = tipos_codificados
          print('Palavra: ' + palavra + ' tipo adicionado: ' + str(tipos_codificados[0]))

        # Caso a consulta tenha retornado mais de 1 dos tipos observados
        # faz uma nova verificação pelo usando mais 1 filtro para a palavra em ingles
        elif len(tipos_codificados) > 1:

          palavra_em_ingles = GoogleTranslator(source='pt', target='en').translate(palavra).capitalize()
          # print(f'palavra: {palavra}  tradução: {palavra_em_ingles}')
          
          query = f'select distinct ?type where {{ ?i rdfs:label "{palavra}"@pt ; a ?type . FILTER (strstarts(str(?type), str("http://dbpedia.org/ontology/"))) FILTER (strstarts(str(?i), str("http://dbpedia.org/resource/{palavra_em_ingles}"))) }}'
      
          sparql.setQuery(query)
          
          # Configura o retorno para o formato JSON
          sparql.setReturnFormat(JSON)
          
          # Realiza a consulta
          dbpedia_return_dict = sparql.query().convert()
          
          # Obtem a lista de "types" retornados pela consulta com filtro da palavra em ingles
          return_list_english = dbpedia_return_dict['results']['bindings']

          if return_list_english:
        
            tipos_dbpedia_retornados_ingles = [result['type']['value'].split('/')[-1] for result in return_list_english]

            tipos_obsdervados_encontrados_ingles = [tipo for tipo in tipos_dbpedia_retornados_ingles if tipo in TIPOS_OBSERVADOS]

            tipos_codificados_ingles = []
            # Adiciona na lista apenas 1 valor de cada tipo observado
            for tipo in tipos_obsdervados_encontrados_ingles:
              if TIPOS_OBSERVADOS[tipo] not in tipos_codificados_ingles:
                tipos_codificados_ingles.append(TIPOS_OBSERVADOS[tipo])
        
            if len(tipos_codificados_ingles) == 1:
              TODOS_OS_TIPOS[palavra] = tipos_codificados_ingles
              print('Palavra: ' + palavra + ' tipo adicionado: ' + str(tipos_codificados_ingles[0]))

            else:
              TODOS_OS_TIPOS[palavra] = tipos_codificados
              print('Palavra: ' + palavra + ' tipo adicionado: ') 
              print(tipos_codificados)

          # Obtem a última palavra do link retornado como "type". Ex: "http://dbpedia.org/ontology/Drug" -> "Drug"
          # dbpedia_type = result['type']['value'].split('/')[-1]
          
          # Verifica se o resultado pertence ao conjunto de tipos observados e se a palavra já tem esse resultado salvo
          # if dbpedia_type in TIPOS_OBSERVADOS and not TIPOS_OBSERVADOS[dbpedia_type] in TODOS_OS_TIPOS[palavra]:
            
            # Adiciona o novo tipo á lista de tipos da palavra
            # TODOS_OS_TIPOS[palavra].append(TIPOS_OBSERVADOS[dbpedia_type])
            # print('Palavra: ' + palavra + ' tipo adicionado: ' + dbpedia_type)

      else:
        # Faz uma segunda busca pela palavra em maiúscula
        palavra_upper = palavra.upper()

        # Query usada para fazer a pesquisa na DBPedia
        query = f'select distinct ?type where {{ ?i rdfs:label "{palavra_upper}"@pt ; a ?type . FILTER (strstarts(str(?type), str("http://dbpedia.org/ontology/"))) }}'
        sparql.setQuery(query)
        
        # Configura o retorno para o formato JSON
        sparql.setReturnFormat(JSON)
        
        # Realiza a consulta
        dbpedia_return_dict = sparql.query().convert()
        
        # Obtem a lista de "types" retornados pela consulta
        return_list_upper = dbpedia_return_dict['results']['bindings']

        # Verifica se existem resultados
        if return_list_upper:
          TODOS_OS_TIPOS[palavra_upper] = []

          tipos_dbpedia_retornados = [result['type']['value'].split('/')[-1] for result in return_list]

          tipos_obsdervados_encontrados = [tipo for tipo in tipos_dbpedia_retornados if tipo in TIPOS_OBSERVADOS]

          tipos_codificados = []
          for tipo in tipos_obsdervados_encontrados:
            if TIPOS_OBSERVADOS[tipo] not in tipos_codificados:
              tipos_codificados.append(TIPOS_OBSERVADOS[tipo])

          if len(tipos_codificados) == 1:
            TODOS_OS_TIPOS[palavra_upper] = tipos_codificados
            print('Palavra: ' + palavra_upper + ' tipo adicionado: ' + str(tipos_codificados[0]))

          # Caso a consulta tenha retornado mais de 1 dos tipos observados
          # faz uma nova verificação pelo usando mais 1 filtro para a palavra em ingles
          elif len(tipos_codificados) > 1:
            palavra_em_ingles = GoogleTranslator(source='pt', target='en').translate(palavra).capitalize()
            # print(f'palavra: {palavra}  tradução: {palavra_em_ingles}')

            query = f'select distinct ?type where {{ ?i rdfs:label "{palavra_upper}"@pt ; a ?type . FILTER (strstarts(str(?type), str("http://dbpedia.org/ontology/"))) FILTER (strstarts(str(?i), str("http://dbpedia.org/resource/{palavra_em_ingles}"))) }}'
      
            sparql.setQuery(query)
          
            # Configura o retorno para o formato JSON
            sparql.setReturnFormat(JSON)
            
            # Realiza a consulta
            dbpedia_return_dict = sparql.query().convert()
            
            # Obtem a lista de "types" retornados pela consulta com filtro da palavra em ingles
            return_list_english = dbpedia_return_dict['results']['bindings']

            if return_list_english:
          
              tipos_dbpedia_retornados_ingles = [result['type']['value'].split('/')[-1] for result in return_list_english]

              tipos_obsdervados_encontrados_ingles = [tipo for tipo in tipos_dbpedia_retornados_ingles if tipo in TIPOS_OBSERVADOS]

              tipos_codificados_ingles = []
              for tipo in tipos_obsdervados_encontrados_ingles:
                if TIPOS_OBSERVADOS[tipo] not in tipos_codificados_ingles:
                  tipos_codificados_ingles.append(TIPOS_OBSERVADOS[tipo])
          
              if len(tipos_codificados_ingles) == 1:
                TODOS_OS_TIPOS[palavra_upper] = tipos_codificados_ingles
                print('Palavra: ' + palavra + ' tipo adicionado: ' + str(tipos_codificados_ingles[0]))

              else:
                TODOS_OS_TIPOS[palavra_upper] = tipos_codificados
                print('Palavra: ' + palavra_upper + ' tipo adicionado: ')
                print(tipos_codificados)
          
          # for result in return_list_upper:

          #   # Obtem a última palavra do link retornado como "type". Ex: http://dbpedia.org/ontology/Drug -> Drug
          #   dbpedia_type = result['type']['value'].split('/')[-1]
          
          #   # Verifica se o resultado pertence ao conjunto de tipos observados e se a palavra já tem esse resultado salvo
          #   if dbpedia_type in TIPOS_OBSERVADOS and not TIPOS_OBSERVADOS[dbpedia_type] in TODOS_OS_TIPOS[palavra_upper]:
              
          #     # Caso encontra resultado para a palavra em Maiúsculo, será criado um novo registro,
          #     # desta forma a palavra pode aparecer 2 vezes com "Cases" diferentes.
          #     TODOS_OS_TIPOS[palavra_upper].append(TIPOS_OBSERVADOS[dbpedia_type])
          #     print('Palavra: ' + palavra_upper + ' tipo adicionado: ' + dbpedia_type)


In [ ]:
def salvaArquivoCsvComOsTiposDBpedia():
  pd.DataFrame.from_dict(data=TODOS_OS_TIPOS, orient='index').to_csv(
      DIRETORIO_LOCAL + ARQUIVO_CSV_TIPOS_DBPEDIA, header=False, sep=";")

  if IN_COLAB:
   
    # Copia o arquivo das postagging original       
    !cp "$DIRETORIO_LOCAL$ARQUIVO_CSV_TIPOS_DBPEDIA" "$DIRETORIO_DRIVE"
        
    logging.info("Terminei a cópia do arquivo.")

# 4 Gerar POS-Tagging

## 4.1 Carregamento dos arquivos de dados

### 4.1.1 Cria o diretório local para receber os dados

In [ ]:
# Biblioteca para acessar o sistema de arquivos
import os

#Cria o diretório para receber os arquivos Originais e Permutados
# Diretório a ser criado
dirbase = DIRETORIO_LOCAL[:-1]

if not os.path.exists(dirbase):  
    # Cria o diretório
    os.makedirs(dirbase)    
    logging.info("Diretório criado: {}.".format(dirbase))
else:    
    logging.info("Diretório já existe: {}.".format(dirbase))

### 4.1.2 Copia os arquivos do Google Drive para o Colaboratory

Copia os arquivos do google drive

In [ ]:
# Se estiver executando no Google Colaboratory
if IN_COLAB:

  !cp "$DIRETORIO_DRIVE$NOMEARQUIVOORIGINALCOMPACTADO" "$DIRETORIO_LOCAL$NOMEARQUIVOORIGINALCOMPACTADO"
  !cp "$DIRETORIO_DRIVE$ARQUIVO_CSV_TIPOS_DBPEDIA" "$DIRETORIO_LOCAL$ARQUIVO_CSV_TIPOS_DBPEDIA"

  logging.info("Terminei a cópia.")

cp: cannot stat '/content/drive/MyDrive/Colab Notebooks/Data/DADOS_NER/tipos_dbpedia.csv': No such file or directory


Descompacta os arquivos.

Usa o unzip para descompactar:
*   `-o` sobrescreve o arquivo se existir
*   `-j` Não cria nenhum diretório
*   `-q` Desliga as mensagens 
*   `-d` Diretório de destino



In [ ]:
# Se estiver executando no Google Colaboratory
if IN_COLAB:
  !unzip -o -j -q "$DIRETORIO_LOCAL$NOMEARQUIVOORIGINALCOMPACTADO" -d "$DIRETORIO_LOCAL"

  logging.info("Terminei a descompactação.")

### 4.1.3 Carregamento das lista com os dados dos arquivos originais e permutados

#### Carrega o arquivo dos dados originais

In [ ]:
# Import das bibliotecas
import pandas as pd

# Abre o arquivo e retorna o DataFrame
lista_documentos_originais = pd.read_csv(DIRETORIO_LOCAL + NOMEARQUIVOORIGINAL, sep=";", encoding="UTF-8")


logging.info("TERMINADO ORIGINAIS: {}.".format(len(lista_documentos_originais)))

In [ ]:
lista_documentos_originais.sample(5)

,id,sentencas,documento
21,54,['1. Oriento consulta presencial com médico em...,1. Oriento consulta presencial com médico em e...
341,889,['orientações gerais sobre biossegurança encam...,orientações gerais sobre biossegurança encam p...
252,644,['Oriento sinais de alarme e retorno /atendime...,Oriento sinais de alarme e retorno /atendiment...
9,27,['1. Solicito RT COVID 2. PRescrevo 05 dias de...,1. Solicito RT COVID 2. PRescrevo 05 dias de r...
114,315,['Oriento atendimento na emergência devido ao ...,Oriento atendimento na emergência devido ao ch...


#### Corrigir os tipos de colunas dos dados originais

Em dados originais:
- coluna 1 - `sentenças` carregadas do arquivo vem como string e não como lista.

In [ ]:
# Import das bibliotecas.
import ast # Biblioteca para conversão de string em lista

# Verifica se o tipo da coluna não é list e converte
lista_documentos_originais["sentencas"] = lista_documentos_originais["sentencas"].apply(lambda x: ast.literal_eval(x) if type(x)!=list else x)

logging.info("TERMINADO CORREÇÃO ORIGINAIS: {}.".format(len(lista_documentos_originais)))

## 4.2 Gerar NER Dados Clínicos

### 4.2.1 Carrega o arquivo com os tipos da DBPedia

In [ ]:
if LER_CSV_TIPOS_DBPEDIA_DO_DRIVE:
  import csv

  try:
    with open(DIRETORIO_LOCAL + ARQUIVO_CSV_TIPOS_DBPEDIA, 'r', encoding="UTF-8") as types_csv_file:  
      for linha in csv.reader(types_csv_file, delimiter=';'):
        palavra = linha[0]
        TODOS_OS_TIPOS[palavra] = linha[1:]
        
    print("Arquivo de tipos DBpedia carregado")
    
  except FileNotFoundError:
    print("Arquivo de Tipos DBpedia não encontrado.")

Arquivo de Tipos DBpedia não encontrado.


In [ ]:
TODOS_OS_TIPOS

{}

### 4.2.2 Gerar NER



In [ ]:
# Import das bibliotecas.
import os # Biblioteca para acessar o sistema de arquivos
from tqdm.notebook import tqdm as tqdm_notebook # Biblioteca para barra de progresso

logging.info("Processando {} documentos originais.".format(len(lista_documentos_originais)))

# Barra de progresso dos dados
dados_bar_ner = tqdm_notebook(lista_documentos_originais.iterrows(), desc=f"Dados", unit=f"registro", total=len(lista_documentos_originais))

# Percorre as linhas dos documentos
for i, linha_documento in dados_bar_ner:

  # Carrega a lista das sentenças do documento de acordo com o tipo armazenado
    lista_sentenca_documento = linha_documento[1]

    # Percorre as sentenças do documento
    for j, sentenca in enumerate(lista_sentenca_documento):

      # Carrega os tipos das palavras da sentença no DBpedia
      getListaTiposDBpedia(sentenca)


Dados:   0%|          | 0/387 [00:00<?, ?registro/s]

Palavra: Mapa tipo adicionado: 4
Palavra: Topiramato tipo adicionado: 1


In [ ]:
salvaArquivoCsvComOsTiposDBpedia()

### 4.2.5 Carrega os dados

Carrega os dados das sentencas a partir dos arquivos.


In [ ]:
# Import das bibliotecas

# Abre o arquivo e retorna o DataFrame
tipos_dbpedia = pd.read_csv(DIRETORIO_LOCAL + ARQUIVO_CSV_TIPOS_DBPEDIA, header=None, na_filter=False, sep=";", encoding="UTF-8")

print(len(tipos_dbpedia))

1001


In [ ]:
tipos_dbpedia.sample(5)

,0,1
633,Cai,
196,Acetilcisteína,1.0
328,Trabalho,
823,Contaminação,
492,Pelo,


# 5 Finalização

## 5.1 Tempo final de processamento



In [ ]:
# Pega o tempo atual menos o tempo do início do processamento.
finalProcessamento = time.time()
tempoTotalProcessamento = formataTempo(finalProcessamento - inicioProcessamento)

print("")
print("  Tempo processamento:  {:} (h:mm:ss)".format(tempoTotalProcessamento))


  Tempo processamento:  5:10:52 (h:mm:ss)
